In [16]:
import netCDF4
import geojson
import numpy as np
import shapely.geometry

In [4]:
ds = netCDF4.Dataset('/Users/baart_f/data/rijnland/vanGovert/Leerdam/leerdamwest_map.nc')

In [22]:

x = ds.variables['mesh2d_face_x'][:]
y = ds.variables['mesh2d_face_y'][:]
ucx = ds.variables['mesh2d_ucx'][-1]
ucy = ds.variables['mesh2d_ucy'][-1]
angles = np.rad2deg(np.arctan2(ucy, ucx))
lengths = np.sqrt(ucx ** 2 + ucy **2 )

In [34]:
crs = geojson.crs.Named(
    properties={
        "name": "urn:ogc:def:crs:EPSG::{:d}".format(28992)
    }
)
features= []
for i, (x_i, y_i, angle_i, length_i, u_i, v_i) in enumerate(zip(x, y, angles, lengths, ucx, ucy)):
    point = shapely.geometry.Point(x_i, y_i)
    geometry = point.__geo_interface__
    geometry['crs'] = dict(crs)
    feature = geojson.Feature(
        id=i, 
        geometry=geometry, 
        properties={
            "angle_east_ccw": angle_i,
            "angle_north_cw": 90 - angle_i,
            "length": length_i,
            "u": u_i,
            "v": v_i
        }
    )
    features.append(feature)


In [35]:
fc = geojson.FeatureCollection(features=features)
with open('quiver.json', 'w') as f:
    geojson.dump(fc, f)
